In [ ]:
!uv add ipykernel
!uv add torch
!uv add transformers
!uv add numpy
!uv add tqdm
!uv add llama_cpp

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import gc

# 清理缓存
torch.cuda.empty_cache()
gc.collect()

# 加载模型和分词器
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct", 
    device_map="cuda",
    dtype="auto", 
    trust_remote_code=False,
)

tokenizer = AutoTokenizer.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct"
)

# 创建文本生成管道
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    temperature=0.9,
    return_full_text=False,
    max_new_tokens=500,
    do_sample=False
)

# 提示词
messages = [
    {"role": "user", "content": "Create a funny joke about chickens."}
]

# 生成输出
output = pipe(messages)
print(output[0]["generated_text"])

# 应用提示词模板
prompt = pipe.tokenizer.apply_chat_template(
    messages, 
    tokenize=False
)
print(prompt)

In [ ]:
# 使用虚构词的单个句子示例
one_shot_prompt = [
    {
        "role": "user",
        "content": "A 'Gigamuru' is a type of Japanese musical instrument. An example of a sentence that uses the word Gigamuru is:"
    },
    {
        "role": "assistant",
        "content": "I have a Gigamuru that my uncle gave me as a gift. I love to play it at home."
    },
    {
        "role": "user",
        "content": "To 'screeg' something is to swing a sword at it. An example of a sentence that uses the word screeg is:"
    }
]
print(tokenizer.apply_chat_template(one_shot_prompt, tokenize=False))

In [ ]:
# 为产品创建名称和口号
product_prompt = [
    {
        "role": "user",
        "content": "Create a name and slogan for a chatbot that leverages LLMs."
    }
]
outputs = pipe(product_prompt)
product_description = outputs[0]["generated_text"]
print(product_description)

In [ ]:
# 基于产品名称和口号生成销售宣传语
sales_prompt = [
    {
        "role": "user",
        "content": f"Generate a very short sales pitch for the following product: {product_description}"
    }
]
outputs = pipe(sales_prompt)
sales_pitch = outputs[0]["generated_text"]
print(sales_pitch)

In [ ]:
# 使用思维链回答
cot_prompt = [
    {"role": "user", "content": "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?"},
    {"role": "assistant", "content": "Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. 5 + 6 = 11. The answer is 11."},
    {"role": "user", "content": "The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?"}
]
# 生成输出
outputs = pipe(cot_prompt)
print(outputs[0]["generated_text"])

In [ ]:
# 零样本思维链
zeroshot_cot_prompt = [
    {"role": "user", "content": "The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have? Let's think step-by-step."}
]
# 生成输出
outputs = pipe(zeroshot_cot_prompt)
print(outputs[0]["generated_text"])

In [ ]:
# 零样本思维树
zeroshot_tot_prompt = [
    {"role": "user", "content": "Imagine three different experts are answering this question. All experts will write down 1 step of their thinking, then share it with the group. Then all experts will go on to the next step, etc. If any expert realizes they're wrong at any point then they leave. The question is 'The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?' Make sure to discuss the results."}
]
# 生成输出
outputs = pipe(zeroshot_tot_prompt)
print(outputs[0]["generated_text"])

In [ ]:
# 零样本学习：不提供示例
zeroshot_prompt = [
    {"role": "user", "content": "Create a character profile for an RPG game in JSON format."}
]
# 生成输出
outputs = pipe(zeroshot_prompt)
print(outputs[0]["generated_text"])

In [ ]:
# 单样本学习：提供输出结构示例
one_shot_template = """Create a short character profile for an RPG game. Make sure to only use this format:
{
  "description": "A SHORT DESCRIPTION",
  "name": "THE CHARACTER'S NAME",
  "armor": "ONE PIECE OF ARMOR",
  "weapon": "ONE OR MORE WEAPONS"
}
"""
one_shot_prompt = [
    {"role": "user", "content": one_shot_template}
]
# 生成输出
outputs = pipe(one_shot_prompt)
print(outputs[0]["generated_text"])

In [ ]:
import gc
import torch

del model, tokenizer, pipe

# 清理缓存和显存
gc.collect()
torch.cuda.empty_cache()

from llama_cpp.llama import Llama

# 加载 Phi-3
llm = Llama.from_pretrained(
    repo_id="microsoft/Phi-3-mini-4k-instruct-gguf",
    filename="*fp16.gguf",
    n_gpu_layers=-1,
    n_ctx=2048,
    verbose=False
)

# 生成输出
output = llm.create_chat_completion(
    messages=[
        {"role": "user", "content": "Create a warrior for an RGP in JSON format."},
    ],
    response_format={"type": "jons_object"},
    temperature=0,
)["choices"][0]['message']['content']

In [ ]:
import json

json_output = json.dumps(json.loads(output), indent=4)
print(json_output)